In [ ]:
import numpy as np
import pandas as pd
import time
import math
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.neighbors.regression import KNeighborsRegressor, check_array, _get_weights

class MedianKNNRegressor(KNeighborsRegressor):
    def predict(self, X):
        X = check_array(X, accept_sparse='csr')

        neigh_dist, neigh_ind = self.kneighbors(X)

        weights = _get_weights(neigh_dist, self.weights)

        _y = self._y
        if _y.ndim == 1:
            _y = _y.reshape((-1, 1))

        ######## Begin modification
        if weights is None:
            y_pred = np.median(_y[neigh_ind], axis=1)
        else:
            # y_pred = weighted_median(_y[neigh_ind], weights, axis=1)
            raise NotImplementedError("weighted median")
        ######### End modification

        if self._y.ndim == 1:
            y_pred = y_pred.ravel()

        return y_pred    

# [  2.38172861  13.3871126    9.6737255    2.77561858  17.07392584]
# # APPLYING PSO

def kNN(X,w,k,Y):                                  ### This function finds mse
  X_v,X_t,Y_v,Y_t=train_test_split(X,Y,test_size=0.2)
  columns=list(X_v)
  X_temp_v=X_v.copy()
  X_temp_t=X_t.copy()
  j=0
  for i  in columns:
    X_temp_t[i]=X_t[i]*w[j]
    X_temp_v[i]=X_v[i]*w[j]
    j+=1
  knn1=KNeighborsRegressor()
  knn1.fit(X_temp_v,Y_v)
  y_t_p=knn1.predict(X_t)
  return mean_squared_error(Y_t,y_t_p,squared=False)
  
# pso is used to find the weights of features.
# returns weight vector
#objective function is the mse of knn
# hyperparameters itr,c1,c2,w,pop_size
def Bound(v,Pop):
  
  
  rows,columns=np.shape(Pop)                       # Weights can only lie between 0 to 1000 (boundaries chosen) 
  
  
  for i in range(rows):
    for j in range(columns):
      if(v[i][j]+Pop[i][j]>100):
        Pop[i][j]=100
      elif(v[i][j]+Pop[i][j]<0):
        Pop[i][j]=0
      else:
        Pop[i][j]=v[i][j]+Pop[i][j]
  
  
  return Pop
def Best(Pop,pbest,X,Y,k):              # Updates the best objective function value for each value and returns that
  rows,columns=np.shape(Pop)
  pnew=list()
  for i in range(rows):
    if(kNN(X,Pop[i],k,Y)<pbest[i]):
      pnew.append(kNN(X,Pop[i],k,Y))
    else:
      pnew.append(pbest[i])
  pN=np.array(pnew)  
  return pN
def pso(w,c1,c2,itr,X,Y,pop_size,k):
  # particle swarm optimization, Pop list contains the particles, the gbest stores global best. Function returns weight vector (particle)
  #corresponding to gbest
  rows,features=np.shape(X)
  Pop=[float(random.random()*10) for i in range(features*pop_size)]  
  P=np.reshape(Pop,(pop_size,features))
  Pop=P
  f=[kNN(X,P[i],k,Y) for i in range(pop_size)]
  # f contains the values of kNN corresponding to each particle taken as weight vector. It is to pop for sorting ease
  P=np.column_stack((P,np.array(f)))
  
  pbest=np.array(f)
  v1=[random.random()*10 for i in range(features*pop_size)]
  v=(np.reshape(v1,(pop_size,features)))
  #w=list()
  P=(P[P[:,features].argsort()])
  #w=P[1][0:features]
  # w stores the weight corresponding to gbest
  # each iteration prints the weight vector and corresponding gbest(lowest mse in that iteration)
  
  print(w)
  gbest=kNN(X,w,k,Y)
  print(gbest)
  for i in range(itr):
    
    R1=[float(random.random()) for k in range(features)]
    R2=[float(random.random()) for k in range(features)]
    r1=np.array(R1)
    r2=np.array(R2)
    
    
    
    print(gbest)
    for j in range(pop_size):
      v[j]=w*v[j]+c1*r1*(pbest[j]-Pop[j])+c2*r2*(gbest-Pop[j]) 
    # velocity is calculated, and then the new particle weights is obtained after checking boundary condition
    
    
    Pop=Bound(v,Pop)
    pbest=Best(Pop,pbest,X,Y,k)
    
    
    for i in range(pop_size):
      if(pbest[i]<gbest):
        gbest=pbest[i]
        w=Pop[i]
    print(w)
  
    
  return w
begin=time.time()
df=pd.read_csv('LaqnData 2014-2015.csv')
df2=pd.read_csv('LaqnDataPart2data.csv')
df3=pd.read_csv('/content/LaqnData  2015-2017.csv')
df4=pd.read_csv('LaqnData 2015-2017Part2.csv')
df5=pd.read_csv('LaqnData 2017-2019.csv')
df6=pd.read_csv('LaqnData 2017-2019Part2.csv')
df1=pd.DataFrame()
BP=[]
#date=[]
NO=[]
NO2=[]
NOX=[]
RAIN=[]
RHUM=[]
SOLR=[]
SO2=[]
WDIR=[]
WSPD=[]
#df[df['Value'] < 0]=float('NaN')
#df2[df2['Value'] < 0]=float('NaN')
j=0
for i in df['Value']:
  if i<0:
    df['Value'][j]=float('NaN')
  j+=1
j=0
for k in df2['Value']:
  if k<0:
    df2['Value'][j]=float('NaN')
  j+=1
for i in df2.index:
    if df2['Species'][i]=='SOLR':
        SOLR.append(df2['Value'][i])
    if df2['Species'][i]=='SO2':
        SO2.append(df2['Value'][i])
    if df2['Species'][i]=='WDIR':
        WDIR.append(df2['Value'][i])
    if df2['Species'][i]=='WSPD':
        WSPD.append(df2['Value'][i])

for i in df.index:
    if df['Species'][i]=='BP':
        BP.append(df['Value'][i])
        
    if df['Species'][i]=='NO':
        NO.append(df['Value'][i])
    if df['Species'][i]=='NO2':
        NO2.append(df['Value'][i])
    if df['Species'][i]=="NOX":
        NOX.append(df['Value'][i])
    if df['Species'][i]=='RAIN':
        RAIN.append(df['Value'][i])
    if df['Species'][i]=='RHUM':
        RHUM.append(df['Value'][i])
j=0
for i in df3['Value']:
  if i<0:
    df3['Value'][j]=float('NaN')
  j+=1
j=0
for k in df4['Value']:
  if k<0:
    df4['Value'][j]=float('NaN')
  j+=1
for i in df4.index:
    if df4['Species'][i]=='SOLR':
        SOLR.append(df4['Value'][i])
    if df4['Species'][i]=='SO2':
        SO2.append(df4['Value'][i])
    if df4['Species'][i]=='WDIR':
        WDIR.append(df4['Value'][i])
    if df4['Species'][i]=='WSPD':
        WSPD.append(df4['Value'][i])

for i in df3.index:
    if df3['Species'][i]=='BP':
        BP.append(df3['Value'][i])
        
    if df3['Species'][i]=='NO':
        NO.append(df3['Value'][i])
    if df3['Species'][i]=='NO2':
        NO2.append(df3['Value'][i])
    if df3['Species'][i]=="NOX":
        NOX.append(df3['Value'][i])
    if df3['Species'][i]=='RAIN':
        RAIN.append(df3['Value'][i])
    if df3['Species'][i]=='RHUM':
        RHUM.append(df3['Value'][i])
j=0
for i in df5['Value']:
  if i<0:
    df5['Value'][j]=float('NaN')
  j+=1
j=0
for k in df6['Value']:
  if k<0:
    df6['Value'][j]=float('NaN')
  j+=1
for i in df6.index:
    if df6['Species'][i]=='SOLR':
        SOLR.append(df6['Value'][i])
    if df6['Species'][i]=='SO2':
        SO2.append(df6['Value'][i])
    if df6['Species'][i]=='WDIR':
        WDIR.append(df6['Value'][i])
    if df6['Species'][i]=='WSPD':
        WSPD.append(df6['Value'][i])

for i in df5.index:
    if df5['Species'][i]=='BP':
        BP.append(df5['Value'][i])
        
    if df5['Species'][i]=='NO':
        NO.append(df5['Value'][i])
    if df5['Species'][i]=='NO2':
        NO2.append(df5['Value'][i])
    if df5['Species'][i]=="NOX":
        NOX.append(df5['Value'][i])
    if df5['Species'][i]=='RAIN':
        RAIN.append(df5['Value'][i])
    if df5['Species'][i]=='RHUM':
        RHUM.append(df5['Value'][i])

#df1['ReadingDateTime']=date
df1['BP']=BP
df1['NO']=NO
df1['NO2']=NO2
df1['NOX']=NOX
df1['RAIN']=RAIN
df1['SOLR']=SOLR
df1['SO2']=SO2
df1['SOLR']=SOLR
df1['WDIR']=WDIR
df1['WSPD']=WSPD

#Removing outliers using boxplot
columns=list(df1)
for i in columns:
  j=0
  Q1=df1[i].quantile(0.25)
  Q3=df1[i].quantile(0.75)
  IQR=Q3-Q1
  for k in df1[i]:
    if(k<Q1-(1.5*IQR) or k>Q3+(1.5*IQR)):
      df1[i][j]=float("NaN")
    j+=1
#print(df1.tail())

df1.interpolate(limit_direction='both',inplace=True)
#df1.to_csv("cleaned.csv")
#scaler=MinMaxScaler()
#df1[['BP','NO2','NO','NOX','SO2','SOLR','WDIR','WSPD']]=scaler.fit_transform(df1[['BP','NO2','NO','NOX','SO2','SOLR','WDIR','WSPD']])
print(df1.tail())

X=df1[['BP','SO2','NO2','NOX','SOLR','WDIR','WSPD']]
Y=df1[['NO']]
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

columns=list(X_train)
knn_normal=KNeighborsRegressor()
knn_normal.fit(X_train,Y_train)
y_p=knn_normal.predict(X_test)
print(mean_squared_error(Y_test,y_p,squared=False))
#print(X_train.tail())
X_temp=X_train.copy()
X_test_temp=X_test.copy()
power=1
w=[]
j=0
for i in columns:
  r,p=pearsonr(X_train[i],Y_train['NO'])
  w.append(round(abs(r)/(1-abs(r)),3))
  X_temp[i]=X_train[i]*w[j]
  X_test_temp[i]=X_test[i]*w[j] 
  j+=1
knn=KNeighborsRegressor()
knn.fit(X_temp,Y_train)
y_predict=knn.predict(X_test_temp)
print(f"MSE of NO for the actual function used {mean_squared_error(Y_test,y_predict,squared=False)}")
X_temp=X_train.copy()
X_test_temp=X_test.copy()
power=1
w=[]
j=0
sum=1
for i in columns:
  r,p=pearsonr(X_train[i],Y_train['NO'])
  print(r)
  w.append(round((pow(abs(r),0.5))/(1-abs(r)),3))
#   sum+=w[j]
  
# for i in columns:
  X_temp[i]=X_train[i]*(w[j])
  X_test_temp[i]=X_test[i]*(w[j]) 
  j+=1
knn=KNeighborsRegressor(algorithm='ball_tree')
knn.fit(X_temp,Y_train)
y_predict=knn.predict(X_test_temp)
print(f"MSE of NO for the new function used {mean_squared_error(Y_test,y_predict,squared=False)}")
#Method-2 Part
w=pso(w,0.0005,0.0002,5,X,Y,10,5)
print(w)
j=0
for i in columns:
  X_temp[i]=X_train[i]*w[j]
  X_test_temp[i]=X_test[i]*w[j]
  j+=1
print(X_train.head())

print(f"Final best MSE {mean_squared_error(Y_test,y_p,squared=False)}")
#Method 2 part
end=time.time()
print(f"Total run time of the program {end-begin}")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:199: SettingWithCopyWarning: 
A value is trying 

            BP   NO   NO2   NOX  RAIN  SOLR  SO2   WDIR  WSPD
175291  1029.0  4.1  12.3  18.6   0.0   5.0  2.3  282.0   0.2
175292  1029.0  4.1  12.3  18.6   0.0   5.0  2.3  282.0   0.2
175293  1029.0  4.1  12.3  18.6   0.0   5.0  2.3  272.0   0.2
175294  1029.0  4.1  12.3  18.6   0.0   5.0  2.3  277.0   0.2
175295  1029.0  4.1  12.3  18.6   0.0   4.0  2.3  268.0   0.2
1.0941090541422702
MSE of NO for the actual function used 0.7425823750725186
0.031187768159600206
-0.03622566469705349
0.5695964550076017
0.7643456234112571
0.012448532995873292
0.06065059690818177
-0.20739979072057857
MSE of NO for the new function used 0.7393478580103232
[0.182, 0.197, 1.754, 3.71, 0.113, 0.262, 0.575]
3.231828298470116
3.231828298470116
[ 0.78678745  7.12319737 17.76974587 25.551529    7.90907074  7.00677014
 11.45068077]
3.0837258193704766
[  1.3702165   12.88140364 100.         100.           9.07698113
   8.13163459  63.98881885]
3.0837258193704766
[  2.17007611  87.05287061 100.         100.      